<a href="https://colab.research.google.com/github/khietvuarong/Coding-Exercise---Prompt-Engineering/blob/main/MinhKhietLe_Coding_Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# -------------------------------
# INSTALL DEPENDENCY
# -------------------------------
!pip install -q google-generativeai

In [56]:
# ======================================
# Exercise 1 – Prompt Chaining
# Customer Support AI (Gemini)
# With Animated Typing + API Check
# ======================================

import google.generativeai as genai
from google.colab import userdata
import threading
import time
import sys

# --------------------------------------
# Configure API
# --------------------------------------
api_key = userdata.get('Ex1SecretKey')
genai.configure(api_key=api_key)

print("🔍 Checking Gemini API connection...\n")

available_models = []

for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        available_models.append(m.name)

if not available_models:
    print("❌ No compatible Gemini models found.")
    raise Exception("No working Gemini models available.")
else:
    model_name = available_models[0]
    print("✅ Gemini API Connected Successfully!")
    print("🚀 Using model:", model_name)

model = genai.GenerativeModel(model_name)


# --------------------------------------
# Typing Animation
# --------------------------------------
def typing_animation(stop_event):
    dots = 0
    while not stop_event.is_set():
        print("\r🤖 " + "." * dots + "   ", end="")
        sys.stdout.flush()
        dots = (dots + 1) % 4
        time.sleep(0.5)
    print("\r" + " " * 30 + "\r", end="")  # clear line


# --------------------------------------
# Gemini Call Wrapper
# --------------------------------------
def ask_gemini(prompt):
    stop_event = threading.Event()
    t = threading.Thread(target=typing_animation, args=(stop_event,))
    t.start()

    try:
        response = model.generate_content(prompt)
        result = response.text.strip()
    finally:
        stop_event.set()
        t.join()

    return result


# --------------------------------------
# STEP 1: Classify Issue
# --------------------------------------
def classify_issue(customer_message):
    prompt = f"""
Classify the issue into ONE category:
Billing
Technical Support
Account Access
Product Information
Complaint
Other

Return ONLY the category name.

Customer:
{customer_message}
"""
    return ask_gemini(prompt)


# --------------------------------------
# STEP 2: Gather Info
# --------------------------------------
def gather_info(issue_category, customer_message):
    prompt = f"""
Issue category: {issue_category}

Ask ONE follow-up question.
Do NOT provide solution yet.
Max 2 sentences.
Be professional and polite.

Customer message:
{customer_message}
"""
    return ask_gemini(prompt)


# --------------------------------------
# STEP 3: Propose Solution
# --------------------------------------
def propose_solution(issue_category, customer_message, additional_info):
    prompt = f"""
Issue category: {issue_category}

Customer message:
{customer_message}

Additional info:
{additional_info}

Provide a clear step-by-step solution.
- Max 5 bullet points
- Be empathetic
- Use concise bullets
"""
    return ask_gemini(prompt)


# --------------------------------------
# STEP 4: Escalation
# --------------------------------------
def escalation_decision(issue_category, solution_text):
    prompt = f"""
Issue category: {issue_category}

Solution:
{solution_text}

Return ONLY:
ESCALATE
or
NO ESCALATION

Escalate if:
- refund over $100
- customer angry
- tech issue unresolved
"""
    return ask_gemini(prompt)


# ======================================
# FULL HUMAN-INPUT FLOW
# ======================================

print("\n=== 🤖 Customer Support AI (Gemini) ===\n")

customer_input = input("Customer: ")

# Step 1
print("\n[Step 1] Classifying issue...")
category = classify_issue(customer_input)
print("✔ Category:", category)

# Step 2
print("\n[Step 2] Gathering more information...")
followup = gather_info(category, customer_input)
print("✔ Follow-up Question:")
print(followup)

# Human reply
additional_info = input("\nCustomer Additional Info: ")

# Step 3
print("\n[Step 3] Generating solution...")
solution = propose_solution(category, customer_input, additional_info)
print("✔ Proposed Solution:")
print(solution)

# Step 4
print("\n[Step 4] Checking escalation rule...")
decision = escalation_decision(category, solution)
print("✔ Escalation Decision:", decision)

print("\n=== ✅ End of Support Flow ===")

🔍 Checking Gemini API connection...

✅ Gemini API Connected Successfully!
🚀 Using model: models/gemini-2.5-flash

=== 🤖 Customer Support AI (Gemini) ===

Customer: Hello

[Step 1] Classifying issue...
✔ Category: Other

[Step 2] Gathering more information...
✔ Follow-up Question:
Hello! How may I assist you today?

Customer Additional Info: 2

[Step 3] Generating solution...
✔ Proposed Solution:
Here's a step-by-step solution:

*   Hello there! We received your message and are ready to help, but we'll need a bit more information to understand your request.
*   Could you please tell us more about what you need assistance with, or what the number "2" refers to?
*   Once we have a clearer idea of your query, we'll be happy to provide a complete solution.

[Step 4] Checking escalation rule...
✔ Escalation Decision: NO ESCALATION

=== ✅ End of Support Flow ===
